In [ ]:
from dotenv import load_dotenv
import os
import boto3
import botocore
import pandas as pd

In [ ]:
load_dotenv()

ACCESS_KEY = os.getenv('AWS_ACCESS_KEY_ID')
SECRET_KEY = os.getenv('AWS_SECRET_ACCESS_KEY')
REGION = os.getenv('REGION')

In [ ]:
client = boto3.client(
    's3',
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET_KEY,
)

In [ ]:
s3 = boto3.resource('s3')

In [ ]:
my_bucket = s3.Bucket('legalthings-datalake')

for file in my_bucket.objects.all():
    print(file.key)

In [ ]:
def with_aws_service(f):
    """Decorate function to add aws service."""
    def wrap(*args, **kwargs):
        load_dotenv()
        ACCESS_KEY = os.getenv('STORAGE_ACCOUNT_NAME')
        SECRET_KEY = os.getenv('STORAGE_ACCOUNT_KEY')

        client = boto3.client(
            's3',
            aws_access_key_id=ACCESS_KEY,
            aws_secret_access_key=SECRET_KEY,
        )
        s3 = boto3.resource('s3')
        kwargs['s3'] = s3
        return f(*args, **kwargs)
    return wrap


def make_folder(path):
    """Create folder."""
    if not os.path.exists(path):
        os.makedirs(path)
        
        
def extract_filename_from_filepath(filepath):
    """Extract filename from full filepath."""
    return (
        re
        .search('/(.*).csv', filepath)
        .group(1)
        .split('/')
    )[-1]


@with_aws_service
def download_csv_files(
    bucket: str,remote: str, local: str, *args, **kwargs
):
    make_folder(os.path.join(local, remote))
    s3 = kwargs['s3']
    
    files = list_csv_files(bucket, remote)
    
    for file in files:
        try:
            file_name = extract_filename_from_filepath(file)
            local_filename = os.path.join(local, remote, file_name)
            s3.Bucket(bucket).download_file(file, local_filename)
        except botocore.exceptions.ClientError as e:
            if e.response['Error']['Code'] == "404":
                print("The object does not exist.")
            else:
                raise
                
                
@with_aws_service
def list_csv_files(
    bucket: str, remote: str, *args, **kwargs
) -> list:
    """Returns list of csv files in remote folder."""
    s3 = kwargs['s3']
    
    files = []
    
    my_bucket = s3.Bucket(bucket)
    
    for file in my_bucket.objects.all():
        if remote in file.key and file.key.endswith('.csv'):
            files.append(file.key)
            
    return files

In [ ]:
download_csv_files(
    bucket='legalthings-datalake',
    remote='datastudio/BVshareholders', 
    local='../data/'
)

In [ ]:
BUCKET_NAME = 'legalthings-datalake'
KEY = 'datastudio/BVshareholders/part-00000-20269f50-2b21-4c2d-b896-6bd87829a261-c000.csv'

s3 = boto3.resource('s3')

try:
    s3.Bucket(BUCKET_NAME).download_file(KEY, '../data/example.csv')
except botocore.exceptions.ClientError as e:
    if e.response['Error']['Code'] == "404":
        print("The object does not exist.")
    else:
        raise

In [ ]:
list_csv_files(
    bucket='legalthings-datalake',
    remote='datastudio/BVprocedures/'
)

In [ ]:
my_bucket = s3.Bucket('legalthings-datalake')

bucket_categories = []

for file in my_bucket.objects.all():
    try:
        directory = file.key.split('/')[0] + '/' + file.key.split('/')[1]
        bucket_categories.append(directory)
    except:
        print('No sub directory')

In [ ]:
df = pd.DataFrame(bucket_categories, columns=['directory'])

In [ ]:
df.groupby('directory').size().head()